In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors

# Sample movie dataset
data = {
    'movie_id': [1, 2, 3, 4, 5],
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E'],
    'genres': ['Action', 'Comedy', 'Drama', 'Action|Drama', 'Comedy|Drama']
}

movies = pd.DataFrame(data)

# Sample user ratings dataset
user_ratings = {
    'user_id': [1, 1, 2, 2, 2],
    'movie_id': [1, 2, 3, 4, 5],
    'rating': [4, 5, 3, 4, 2]
}

ratings = pd.DataFrame(user_ratings)

# Content-Based Filtering (using movie genres)
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['genres'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Collaborative Filtering (using Nearest Neighbors)
knn_model = NearestNeighbors(n_neighbors=2, metric='cosine')
knn_model.fit(tfidf_matrix)

# Hybrid Recommender
def hybrid_recommendations(movie_title, top_n=3):
    # Find the movie in the dataset
    movie_idx = movies.index[movies['title'] == movie_title].tolist()[0]

    # Content-Based Recommendations
    content_scores = list(enumerate(cosine_sim[movie_idx]))
    content_scores = sorted(content_scores, key=lambda x: x[1], reverse=True)
                                                          [1:top_n + 1]

    # Collaborative Filtering Recommendations
    _, cf_indices = knn_model.kneighbors(tfidf_matrix[movie_idx],
                                         n_neighbors=top_n + 1)
    cf_indices = cf_indices[0][1:]

    # Combine both recommendations
    hybrid_movie_indices = list(set(cf_indices).union([x[0]
                                                       for x in content_scores]))

    return movies['title'].iloc[hybrid_movie_indices]

# Example usage
movie_title = 'Movie A'
recommendations = hybrid_recommendations(movie_title)
print(f"Top recommendations for '{movie_title}':")
for i, movie in enumerate(recommendations):
    print(f"{i + 1}. {movie}")


Top recommendations for 'Movie A':
1. Movie B
2. Movie C
3. Movie D
